<h1> <center> AMOD-5610: Big Data Major Research Paper </center> </h1>
<h2> <center> Fraud Detection in Financial Transactions </center> </h2>

<br>


Submitted By:-

**Jasmeet Singh Saini** (**0758054**)

**Ayush Sharma**  (**0774583**)


### Import necessary libraries


In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Load the dataset


In [2]:
def load_dataset(file_path):
    try:
        # Load the dataset
        dataset = pd.read_csv(file_path)
        print("Dataset loaded successfully.")
        return dataset
    except Exception as e:
        print(f"Error loading the dataset: {e}")
        return None

Here, the data was loaded.

In [4]:
file_path = 'financial_transactions_dataset.csv'
df         = load_dataset(file_path)

Dataset loaded successfully.


### Dataset

Display the first few rows of the dataset to get an overview

In [5]:
df.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


### Rows and Columns

There are **XYZ rows** and **ABC columns** present in data set i.e., before cleaning.

In [ ]:
df.shape

### Checking columns names

In [ ]:
df.columns

### Rename the Columns

Changing the names of the columns for better understanding.

In [ ]:
# Define a dictionary with the mapping of old column names to new column names (for better naming convention).
column_mapping = {
    'User_ID'                    : 'user_id',
    'Product_ID'                 : 'product_id',
    'Gender'                     : 'gender',
    'Age'                        : 'age',
    'Occupation'                 : 'occupation'
}

# Rename the columns in the DataFrame
df.rename(columns = column_mapping, inplace = True)

# Display the updated DataFrame
df.head(5)


## Exploratory Data Analysis